## Objetivo: testar a rede neural treinada com fator upscale igual a 15.

Passo 1: Imports, definições de caminhos, classes e funções utilizadas pelo programa

In [1]:
# Imports necessários
import os
import PIL
import math
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory

from IPython.display import display


def constroi_modelo(upscale_factor=15, channels=1):
    conv_args = {
        "activation": "relu",
        "kernel_initializer": "Orthogonal",
        "padding": "same",
    }
    inputs = keras.Input(shape=(None, None, channels))
    x = layers.Conv2D(64, 5, **conv_args)(inputs)
    x = layers.Conv2D(64, 3, **conv_args)(x)
    x = layers.Conv2D(32, 3, **conv_args)(x)
    x = layers.Conv2D(channels * (upscale_factor ** 2), 3, **conv_args)(x)
    outputs = tf.nn.depth_to_space(x, upscale_factor)
    
    model = keras.Model(inputs, outputs)
    
    loss_fn = keras.losses.MeanSquaredError()
    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(
        optimizer=optimizer, loss=loss_fn,
    )
    return model


def conversor_alta_resolucao(modelo, img):
    ycbcr = img.convert("YCbCr")
    img_cinza, canal_cb, canal_cr = ycbcr.split()
    img_cinza = img_to_array(img_cinza)
    img_cinza = img_cinza.astype("float32") / 255.0

    entrada = np.expand_dims(img_cinza, axis=0)
    saida = modelo.predict(entrada)

    saida_img_cinza = saida[0]
    saida_img_cinza *= 255.0

    saida_img_cinza = saida_img_cinza.clip(0, 255)
    saida_img_cinza = saida_img_cinza.reshape(
        (np.shape(saida_img_cinza)[0], 
         np.shape(saida_img_cinza)[1])
    )

    saida_img_cinza = PIL.Image.fromarray(np.uint8(saida_img_cinza), mode="L")
    saida_canal_cb = canal_cb.resize(saida_img_cinza.size, PIL.Image.BICUBIC)
    saida_canal_cr = canal_cr.resize(saida_img_cinza.size, PIL.Image.BICUBIC)
    img_resultado = PIL.Image.merge("YCbCr", 
            (saida_img_cinza, saida_canal_cb, saida_canal_cr)).convert(
        "RGB"
    )
    return img_resultado


def abreImagem(nome):
    return PIL.Image.open(nome)


def printImagem(modelo, nome):
    imgBaixa = abreImagem(nome)
    display(imgBaixa)
    
    imgAlta = conversor_alta_resolucao(modelo, imgBaixa)
    display(imgAlta)
    
    print(f"Imagem baixa: {imgBaixa.size[0], imgBaixa.size[1]}")
    print(f"Imagem alta: {imgAlta.size[0], imgAlta.size[1]}")

    
# Pasta com os checkpoints
PASTA_CHECKPOINT: str = r"/home/dangelo/dev/TCC/image-resolution/src/criacao_modelo/checkpoint_15x"
ARQUIVO_CHECKPOINT: str = os.path.join(PASTA_CHECKPOINT, "meu-checkpoint")
PASTA_IMGS: str = r"/home/dangelo/Downloads/imgs_teste"

2021-09-01 17:06:55.226797: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-01 17:06:55.226834: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Construindo o modelo
modelo = constroi_modelo(upscale_factor=15, channels=1)
modelo.load_weights(ARQUIVO_CHECKPOINT)
modelo.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d (Conv2D)              (None, None, None, 64)    1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, None, None, 64)    36928     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 32)    18464     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 225)   65025     
_________________________________________________________________
tf.nn.depth_to_space (TFOpLa (None, None, None, 1)     0         
Total params: 122,081
Trainable params: 122,081
Non-trainable params: 0
_______________________________________________________

2021-09-01 17:13:27.161821: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-01 17:13:27.161847: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-01 17:13:27.161869: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dangelo-mint): /proc/driver/nvidia/version does not exist
2021-09-01 17:13:27.162086: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import sys
sys.getsizeof(modelo)

48

In [ ]:
lista_imagens = os.listdir(PASTA_IMGS)
contador = 0

# Mudando o nome das HR para 3x e fazendo com x15
for img in lista_imagens:
    
    caminho_img: str = os.path.join(PASTA_IMGS, img)
    
    # Se tiver "hr" no nome, mudar para "hr_3x"
    if "hr" in caminho_img:
        caminho_img_3x: str = caminho_img.replace("hr", "hr_3x")
        os.rename(src=caminho_img, dst=caminho_img_3x)
    
    # Se não tiver, quer dizer que será "lr". Processar a img
    elif "lr" in caminho_img:
        caminho_img_15x: str = caminho_img.replace("lr", "hr_15x")
        
        img_lr = abreImagem(caminho_img)
        img_hr_15x = conversor_alta_resolucao(modelo, img_lr)
        
        img_hr_15x.save(caminho_img_15x)
        
    contador += 1
    print(f"Processo executado! {contador} de {len(lista_imagens)} imagens processadas...")

Processo executado! 1 de 96 imagens processadas...
Processo executado! 2 de 96 imagens processadas...
Processo executado! 3 de 96 imagens processadas...
Processo executado! 4 de 96 imagens processadas...
Processo executado! 5 de 96 imagens processadas...
Processo executado! 6 de 96 imagens processadas...
Processo executado! 7 de 96 imagens processadas...


2021-09-01 17:42:35.653937: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1323450900 exceeds 10% of free system memory.


Processo executado! 8 de 96 imagens processadas...
Processo executado! 9 de 96 imagens processadas...
Processo executado! 10 de 96 imagens processadas...
Processo executado! 11 de 96 imagens processadas...
Processo executado! 12 de 96 imagens processadas...
Processo executado! 13 de 96 imagens processadas...
Processo executado! 14 de 96 imagens processadas...
Processo executado! 15 de 96 imagens processadas...
Processo executado! 16 de 96 imagens processadas...
Processo executado! 17 de 96 imagens processadas...
Processo executado! 18 de 96 imagens processadas...
Processo executado! 19 de 96 imagens processadas...
Processo executado! 20 de 96 imagens processadas...
Processo executado! 21 de 96 imagens processadas...
Processo executado! 22 de 96 imagens processadas...
Processo executado! 23 de 96 imagens processadas...
Processo executado! 24 de 96 imagens processadas...
Processo executado! 25 de 96 imagens processadas...
Processo executado! 26 de 96 imagens processadas...
Processo execu